# Entendendo a geração de próximos tokens, diferenças de modelos e devices + relembrando aula passada


In [ ]:
# vendo se temos GPU disponivel

# !nvidia-smi <-- não vai funcionar se não tiver GPU

import numpy as np
import torch
import transformers

torch.cuda.is_available() # retorna Ture se encontrou uma GPU para mandar os jobs

# identificando o dispositivo dinamicamente
device = 'cuda' if torch.cuda.is_available() else 'cpu'

print(device)


Criação do modelo de linguagem Llama

In [ ]:
tiny_llama = transformers.pipeline(
    task = "text-generation",
    model="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    device=device
)

print(tiny_llama)
print(tiny_llama.model)

In [ ]:
prompt = 'today is a beautiful day to'

# Maneira mais direta possivel de usar um modelo. Ele vai usar os parametros padrões para gerar a saida
output = tiny_llama(prompt)

# Se não especificarmos o máximo de novos tokens ele vai gerar ate encontrar um token que sinalisa fim da sentenca.

print(output)


[{'generated_text': 'today is a beautiful day to be alive.\n\n(The camera pans out to show the sun shining brightly, the sky turning from blue to pink, and the leaves rustling in the wind.)\n\nNarrator: And as we look around, we see the world around us in a new light.\n\n(The camera pans out to show the city skyline, the buildings glowing in the sunlight, and the people walking around, smiling and laughing.)\n\nNarrator: We see the beauty of the world around us, and we know that we are lucky to be alive.\n\n(The camera pans out to show the sun setting behind the mountains, the sky turning from pink to orange, and the stars twinkling in the night sky.)\n\nNarrator: And as we watch the sun set, we know that we are witnessing a moment in time that will never be repeated.\n\n(The camera pans out to show the sky turning from orange to red, and the sun setting behind the mountains, the sky turning into a beautiful shade of purple.)\n\nNarrator: We see the beauty of the world around us, and w

# Vendo o processo de geracão token by token

# O que é um Tensor

Um tensor é uma estrutura de dados fundamental utilizada em redes neurais e aprendizado de máquina, representando dados multidimensionais de forma eficiente. Pode ser visto como uma generalização de matrizes, que permite representar dados em várias dimensões (escalar, vetor, matriz, etc.).

Em frameworks como PyTorch (pt) e TensorFlow (tf), tensores são a base para armazenar e manipular dados de entrada, pesos, saídas e gradientes durante o treinamento e inferência de modelos de machine learning. Eles suportam operações matemáticas complexas que são otimizadas para hardware acelerado, como GPUs.

Uma característica importante dos tensores é o método .to(device), que permite mover o tensor para o dispositivo desejado (CPU ou GPU), garantindo que os dados estejam no mesmo dispositivo que o modelo, facilitando a execução eficiente de operações.

In [ ]:
prompt_input_ids = tiny_llama.tokenizer.encode(prompt, return_tensors='pt').input_ids.to(device)

# para inspecionar a geração de tokens, precisamos gerar o output com novos parametros

output = tiny_llama.model.generate(
    prompt_input_ids,
    max_new_tokens = 25,
    return_dict_in_generate = True, # retorne um dicionario com informacoes da geracao de dados
    output_scores = True, # retorne as pontuacoes de cada token gerado
)

print(output.keys())

In [ ]:
# vamos pegar os TRANSITION SCORES (probabildidade da palavra utilizada)

transitions = tiny_llama.model.compute_transition_scores(
    output.sequences,
    output.scores,
    normalize_logits=True
)

In [ ]:
# Ver a sequência de transicão do que foi gerado (ignora o prompt original)

tamanho_prompt = len(prompt_input_ids[0])
print(tamanho_prompt)

# nosso prompt faz parte da resposta gerada
print(output.sequences[0][:tamanho_prompt])


generated_tokens = output.sequences[0][tamanho_prompt:]
print(generated_tokens)

print('| token id | score | token str | pro %')
for (token, score) in zip(generated_tokens,transitions[0]):
    print(token, score.to('cpu').numpy(), tiny_llama.tokenizer.decode(token),np.exp(score.to('cpu').numpy()))

In [ ]:
# Vendo o tempo de execução para gerar um prompt

start = time.time()

output = tiny_llama(prompt, max_new_tokens=100)

end = time.time()

print(tiny_llama.model)
print(f'Tempo de execucao: {end - start:.2f} segundos')
print(output)
print(output[0]['generated_text'])

# Fazendo com que o modelo lembre da conversa e dos prompts anteriores

In [ ]:
prompt1 = "What day is today?"
prompt2 = "What day is tomorrow?"

output = tiny_llama(prompt1, max_new_tokens=10)
print(output[0]['generated_text'])

print("-"*80)

output = tiny_llama(prompt2, max_new_tokens=10)
print(output[0]['generated_text'])

# Ué, o que aconteceu? Alucinou?

In [ ]:
# para se lembrar, precisamos alimentar ele com TUDO QUE FOI GERADO ANTERIORMENTE

prompt1 = "What day is today?"
prompt2 = "What day is tomorrow?"

output = tiny_llama(prompt1, max_new_tokens=10)
print(output[0]['generated_text'])

print("-"*80)

output = tiny_llama(output[0]['generated_text'] + prompt2, max_new_tokens=30)
print(output[0]['generated_text'])

# Carregando outro modelo

O código abaixo vai explodir a memória ram (porque já temos o outro modelo carregado nela).

# Novo Modelo: Falcon-7b

O Falcon-7B é um modelo de linguagem desenvolvido para entender e gerar texto de maneira semelhante aos modelos GPT, com uma arquitetura e escopo intermediários, voltado para uma ampla gama de aplicações em processamento de linguagem natural. Ele é projetado para oferecer um bom equilíbrio entre desempenho e eficiência, sendo adequado tanto para pesquisa quanto para aplicações em ambientes de produção que podem acomodar modelos de médio porte.

Esse modelo é ideal para desenvolvedores e pesquisadores que procuram um modelo poderoso, mas que ainda pode ser gerenciado com recursos computacionais moderados, oferecendo um excelente compromisso entre qualidade e custo operacional.

[Model Link](https://huggingface.co/tiiuae/falcon-7b)

Características


| Setting         | Description        |
| --------------- | -----------------  |
| Parameters      | 7B                 |
| Layers          | 32                 |
| Heads           | 48                 |
| Query Groups    | 1                  |
| Embedding Size  |4544                |
| Batch Size      | 4 million tokens   |
| Total Tokens    | 1.5 trillion       |
| Hardware        | 4 A100-40G GPUs    |





In [ ]:
# deletando e limpando a memória

import gc # garbage colector
del tiny_llama
gc.collect()
torch.cuda.empty_cache()

falcon = transformers.pipeline(
     task='text-generation',
     model='tiiuae/falcon-7b',
     device=device
 )